In [1]:
from flask import Flask, request, jsonify, redirect, url_for
import Data_Validation_Functions as f
import numpy as np,pandas as pd
import time,requests
import os,re

import json,ast# ast包可以将字符串转换为有效字典
import hashlib,base64,hmac
import pymysql as m

In [127]:
#获取token
url = 'https://oapi.dingtalk.com'
param_dict = {
    'appkey':"dingivx2npvflqcop1ag",
    'appsecret': "eLNrFcBcaE65Qn6GvFtRX1f8aJ8aHmFlpN93fZEEzbwM0ItFCpNBENSF3vGSLWDb",
}
access_token = requests.get(url + '/gettoken',param_dict).json()['access_token']
access_token

'369a301aacdd35b6aca1bec06e8b4fd8'

# 一、直接调用在职员工的userlist借口（没用）

In [89]:
#查在职员工
userid=[]
try:
    for i in range(1000):
        param_dict = {
            'access_token': access_token,
            "offset":i,
            "size":50,
            "status_list":"2,3,5,-1"
        #     'department_id': depart_id
        }
        result = requests.post(url + '/topapi/smartwork/hrm/employee/queryonjob',param_dict).json() #/auth/scopes
        userid = userid + result['result']['data_list']
except:
    print('Finish')

userid = pd.Series(userid).unique().tolist()

In [94]:
param_dict = {
    'access_token': access_token,
    'agentid':'1066509961',
    'userid_list':'302421125238151255'
}
requests.post(url + '/topapi/smartwork/hrm/employee/v2/list',param_dict).json()#topapi/smartwork/hrm/employee/list

{'errcode': 400125,
 'errmsg': '企业没有开通该微应用',
 'success': False,
 'request_id': 'o5g2g1sumtmt'}

# 二、先找到部门list，再按部门查找所有员工userid，最后再得出工号对应关系

In [4]:
update = user_update()
update.get_token()
update.get_dept()
param = {:,'dept_id':'356141451'}
result = requests.get(update.url + '/topapi/user/listid',param_ding).json()['result']
self.ding_list = self.ding_list + result["userid_list"]

In [12]:

param_dict = {
    'access_token': update.access_token,
    'agentid':'1066509961',
    'userid_list':'0345293634657745',
    'field_filter_list':'' #工号的字段编号
}


requests.post('https://oapi.dingtalk.com/topapi/smartwork/hrm/employee/v2/list',param_dict).json()



In [14]:
staff_info

{'errcode': 0,
 'errmsg': 'ok',
 'result': [{'corp_id': 'ding84406897bb5bc29c35c2f4657eb6378f',
   'field_data_list': [{'field_code': 'sys01-positionLevel',
     'field_name': '岗位职级',
     'field_value_list': [{'item_index': 0}],
     'group_id': 'sys01'},
    {'field_code': 'sys00-tel',
     'field_name': '分机号',
     'field_value_list': [{'item_index': 0, 'label': '', 'value': ''}],
     'group_id': 'sys00'},
    {'field_code': 'sys05-nowContractEndTime',
     'field_name': '现合同到期日',
     'field_value_list': [{'item_index': 0}],
     'group_id': 'sys05'},
    {'field_code': 'sys01-regularTime',
     'field_name': '实际转正日期',
     'field_value_list': [{'item_index': 0}],
     'group_id': 'sys01'},
    {'field_code': 'sys00-remark',
     'field_name': '备注',
     'field_value_list': [{'item_index': 0, 'label': '', 'value': ''}],
     'group_id': 'sys00'},
    {'field_code': 'sys00-position',
     'field_name': '职位',
     'field_value_list': [{'item_index': 0, 'label': '人事经理', 'value': '人事经

In [3]:
class user_update:
    def __init__(self):
        self.appkey = "dingivx2npvflqcop1ag"       #智能人事应用
        self.appsecret = "eLNrFcBcaE65Qn6GvFtRX1f8aJ8aHmFlpN93fZEEzbwM0ItFCpNBENSF3vGSLWDb"
        self.url = 'https://oapi.dingtalk.com'
        self.access_token = ''
        
        self.dept_list = []
        self.ding_list = []
        self.workno_dict = {}
    
    def get_token(self):          ### 钉钉接口1：获取access_token
        param_dict = {'appkey':self.appkey,'appsecret': self.appsecret}
        self.access_token = requests.get(self.url + '/gettoken',param_dict).json()['access_token']

    def get_dept(self):          ### 钉钉接口2：递归所有部门
        param_dept = {'access_token': self.access_token}
        self.dept_list = requests.get(self.url + '/department/list',param_dept).json()['department'] 

    def get_dingdingid(self):    ### 钉钉接口3：按部门查找所有员工
        for dept_id in self.dept_list:
            param_ding = {'access_token': self.access_token,'dept_id':dept_id['id']}
            result = requests.get(self.url + '/topapi/user/listid',param_ding).json()['result']
            self.ding_list = self.ding_list + result["userid_list"]
            
    def get_workno(self):        ### 钉钉接口4：从员工花名册获取工号
        param_dict = {
            'access_token': self.access_token,
            'agentid':'1066509961',
            'userid_list':'',
            'field_filter_list':'sys00-jobNumber' #工号的字段编号
        }
        for ding_id in update.ding_list:
            try:
                param_dict['userid_list'] = ding_id
                staff_info = requests.post(update.url + '/topapi/smartwork/hrm/employee/v2/list',param_dict).json()
                dingding_id = staff_info['result'][0]['userid']
                workno = staff_info['result'][0]['field_data_list'][0]['field_value_list'][0].get('value')
                update.workno_dict[dingding_id] = workno
            except:
                try:
                    param_dict['userid_list'] = ding_id
                    staff_info = requests.post(update.url + '/topapi/smartwork/hrm/employee/v2/list',param_dict).json()
                    dingding_id = staff_info['result'][0]['userid']
                    workno = staff_info['result'][0]['field_data_list'][0]['field_value_list'][0].get('value')
                    update.workno_dict[dingding_id] = workno
                except:
                    pass
            
    def run(self):
        try:
            self.get_token()
            self.get_dept()
            self.get_dingdingid()
        except:
            f.print_log('API 1 Error')
        try:
            self.get_workno()
        except:
            f.print_log('API 2 Error')
    
class combine_sql:
    def __init__(self,workno_dict):
        self.workno_dict = workno_dict
        self.user_info = None
        self.users = None
        self.table_merge = None

        self.dtype = {}
        self.sql_ck = ''
    
    def ueser_info_ck(self):          ## 1.Clickhouse-员工表取数
        sql_ck = '''select staffid,name,workno,title,organid,deptid,status,is_operative,loaddate
        ,(case when organid like '015807%' or organid like '0159010102%' or organid like '02620203%' or staffid='niejun'
        or (title='总监' and type in (1,2)) then 1 else 0 end) is_allpermission
        from dcp_dw_dmp.dim_staff_info where company=1 and status=1 '''
        columns_ck = ['staffid','name','workno','title','organid','deptid','status','is_operative','loaddate','is_allpermission']
        self.user_info = pd.DataFrame(f.query(sql_ck),columns=columns_ck)
    
    def merge(self):                 ## 2.将dingdingid 关联过去ck查到的员工信息
        columns_ding = ['dingdingid','workno'] 
        self.users = pd.DataFrame(self.workno_dict.items(),columns=columns_ding)

        self.table_merge = pd.merge(self.users,self.user_info,how='inner',on='workno').drop_duplicates()
        self.table_merge = self.table_merge[['dingdingid', 'staffid', 'name', 'workno', 'title', 'organid', 'deptid',
        'status','is_operative','is_allpermission', 'loaddate']]
    
    def dtype_determine(self):      ## 3.判断字段类型，用于拼接values
        for column_sq,column in enumerate(self.table_merge.iloc[1,:]):
            self.dtype[column_sq] = type(column)
            
    def values_combine(self):       ## 4.利用List的join方法拼接 values
        insert_content=[]
        for row in np.array(self.table_merge):
            empty = []
            for num,content in enumerate(row):
                if self.dtype.get(num)==str and content is None:
                    empty.append('\'\'')
                elif self.dtype.get(num)==str:
                    empty.append('\''+content+'\'')
                else:
                    empty.append(str(content))
            insert_content.append(','.join(empty))

        self.sql = r'insert into T_Robot_Dim_User values (' + r'),('.join(insert_content) + r');'
    
    def sql_exacute(self):          ## 5.执行插入语句
        conn = m.connect(host = '172.16.92.96',port=3306,user = 'yonghongbi',password='yonghongbi123',database = 'BIDB')
        cur = conn.cursor()
        try:
            cur.execute('truncate table T_Robot_Dim_User') 
            conn.commit() # 提交到数据库执行
        except:
            conn.rollback() # 如果发生错误则回滚
        try:
            cur.execute(self.sql)
            conn.commit() 
        except:
            conn.rollback() 
        conn.close()
        
    def run(self):
        try:
            self.ueser_info_ck()
            self.merge()
        except:
            f.print_log('Clickhouse Query Error')
            
        try:
            self.dtype_determine()
            self.values_combine()
        except:
            f.print_log('Combine Error')
            
        try: 
            self.sql_exacute()
        except: 
            f.print_log('Insert Error')

In [3]:
start_time1 = time.time()

update = user_update()
update.run()

end_time1 = time.time()
timer = round((end_time1-start_time1)/60,2)
print(timer)

12.2


In [14]:
combiner.sql

"insert into T_Robot_Dim_User_Backup values ('142955348','hulidan','胡丽丹','032636','职员','0160150104','17493',1,1,0,'2021-01-25 04:37:19.333'),('172940223721407979','yeziyi','叶子毅','LSZ21437','职员','0160100201','10851',1,1,0,'2021-01-25 04:37:19.333'),('0423251947938287','wangbo101','王博','031897','职员','021402','17171',1,1,0,'2021-01-25 04:37:19.333'),('083354092724281008','xuxiaopeng','徐小鹏','LSZ29201','职员','0214030101','16567',1,1,0,'2021-01-25 04:37:19.333'),('125890283','mengyajie','孟雅杰','033703','职员','02140401','17474',1,1,0,'2021-01-25 04:37:19.333'),('1940196932058475','luojimei','罗基美','LSZ30847','职员','02140301','12550',1,1,0,'2021-01-25 04:37:19.333'),('185668602721032478','liusiyan','刘斯燕','033338','职员','021402','17171',1,1,0,'2021-01-25 04:37:19.333'),('144167186','lixiu1','李秀','032981','职员','021402','17171',1,1,0,'2021-01-25 04:37:19.333'),('0407644764705078','wudi1','吴迪','LSZ28162','职员','0214030101','16567',1,1,0,'2021-01-25 04:37:19.333'),('315954611236398173','guoyanzhi','郭妍芝','

In [12]:
start_time2 = time.time()

combiner = combine_sql(update.workno_dict)
combiner.run()

end_time2 = time.time()
timer = round((end_time2-start_time2)/60,2)
print(timer)

0.01


## 二、会话

In [154]:
param_dict = {
    'access_token': access_token,
    'sender':'1066509961',
    'cid':'0b84ed96848a3a4b9c50a28b648a6103',
    'msg':json.dumps({"msgtype":"text","text":{"content":"请提交日报。"}})
}

requests.post(url + '/message/send_to_conversation',json.dumps(param_dict)).json()

{'errcode': 40014, 'errmsg': '不合法的access_token'}